In [2]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer

# Use Hugging Face-hosted model (works on Spaces)
MODEL_PATH = "microsoft/phi-1_5"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
llm_model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, trust_remote_code=True)

# Generate response
def generate_answer(question, model):
    try:
        prompt = (
            "You are a compassionate and informative AI health assistant.\n"
            "Answer the following health-related question clearly and supportively. Do not include any signature or names at the end:\n\n"
            f"{question.strip()}\n\n"
            "Answer:"
        )

        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        output_ids = model.generate(
            input_ids,
            max_new_tokens=400,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_ids = output_ids[0][len(input_ids[0]):]
        answer = tokenizer.decode(generated_ids, skip_special_tokens=True)

        return answer.strip()
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# Gradio chatbot function
def chatbot(question):
    return generate_answer(question, llm_model)

# Gradio Interface
if __name__ == '__main__':
    interface = gr.Interface(
        fn=chatbot,
        inputs=gr.Textbox(
            label="Your Question",
            placeholder="Ask a health-related question...",
            lines=4,
        ),
        outputs=gr.Textbox(
            label="AI Health Assistant Response",
            lines=8,
        ),
        title="🩺 AI Health Assistant",
        description=(
            "Type any general health-related question below and receive helpful guidance from the AI assistant.\n"
            "⚠️ Note: This assistant is for informational purposes only and is **not** a substitute for professional medical advice."
        ),
        theme="soft",
        allow_flagging="never",
        live=True,
    )
    interface.launch()


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
